In [1]:
import sys; sys.path.insert(0, '../..') # So that we import the local copy of pyzx if you have installed from Github
import random
import pyzx as zx
from copy import deepcopy
import json

In [ ]:

#random.seed(1337)
circ = zx.generate.cliffordT(5,20)
circ = zx.Circuit.load('mod5d2.tfc')
print(circ.depth())

zx.draw_matplotlib(circ,labels=True,h_edge_draw='box')


In [ ]:
g = circ.copy().to_graph()
zx.clifford_simp(g)

In [ ]:
#g.normalize() # Reposition nodes horizontally to look nicer
zx.draw(g) # This draw function by default uses the D3 Javascript library to draw the diagram. If it doesn't work, use draw_matplotlib() instead


In [ ]:
from ipywidgets import widgets
from IPython.display import display, Markdown

from pyzx.simplify import clifford_iter

graph = circ.copy()
graphs = [zx.draw_matplotlib(graph)]
names = ["start"]
for gs, n in clifford_iter(graph):
    gs.normalize()
    graphs.append(zx.draw_matplotlib(gs))
    names.append(n)
zx.drawing.pack_circuit_nf(gs,'grg')
graphs.append(zx.draw_matplotlib(gs))
names.append("reposition")

def plotter(rewrite):
    display(Markdown("Rewrite step: " + names[rewrite]))
    display(graphs[rewrite])

w = widgets.interactive(plotter, rewrite=(0,len(graphs)-1))
slider = w.children[0]
slider.layout.width = "{!s}px".format(min(800,50*len(graphs)))
output = w.children[-1]
output.layout.height = "{!s}px".format(200+graph.qubit_count()*20)
slider.value = 0
w


In [ ]:
circ2 = g.copy()
circ2.normalize()
circ2 = zx.extract_circuit(circ2)
print(circ2.depth())
zx.draw(circ2)


In [2]:
#random.seed(1337)
g = zx.generate.cliffords(5,20)
zx.simplify.spider_simp(g)
zx.simplify.to_gh(g)
json.dumps([1,2,3])

spider_simp: 9. 7. 3. 1. 1.  5 iterations


'[1, 2, 3]'

In [3]:
import sys; sys.path.insert(0, '../..') # So that we import the local copy of pyzx if you have installed from Github
import random
import pyzx as zx
from copy import deepcopy
import json

#random.seed(1337)
g = zx.generate.cliffords(5,20)
zx.simplify.spider_simp(g)
zx.simplify.to_gh(g)
json.dumps([1,2,3])



from copy import deepcopy

# pyzx match ids

min_depth = 10000

queue = [g]
visited = set()
while len(queue) != 0:
    g1 = queue.pop(-1)

    print("Queue length:", len(queue))

    ids = zx.rules.match_ids_parallel(g1) # (identity_vertex, neighbor1, neighbor2, edge_type)
    lcomps = zx.rules.match_lcomp_parallel(g1) # (V,V)
    pivots = zx.rules.match_pivot_parallel(g1)# + zx.rules.match_pivot_boundary(g1) + zx.rules.match_pivot_gadget(g1) # ((V,V), (V,V))
    # print("ids", ids)
    # print("lcomps", lcomps)
    # print("pivots", pivots)

    for id_match in ids:
        g2 = deepcopy(g1)
        etab, rem_verts, rem_edges, check_isolated_vertices = zx.rules.remove_ids(g2, [id_match])
        
        try:
            before = g2.num_vertices()
            g2.add_edge_table(etab)
            g2.remove_edges(rem_edges)
            g2.remove_vertices(rem_verts)
            if check_isolated_vertices:
                g2.remove_isolated_vertices()
            queue.append(g2)
            after = g2.num_vertices()
            print("id ", before, after)

            _g2 = deepcopy(g2)
            circ = zx.extract_circuit(_g2)
            circ = zx.basic_optimization(circ)
            if circ.depth() < min_depth:
                min_depth = circ.depth()
                print(min_depth)
        except Exception as e:
            pass
    for pivot_match in pivots:
        g2 = deepcopy(g1)
        etab, rem_verts, rem_edges, check_isolated_vertices = zx.rules.pivot(g2, [pivot_match])
        try:
            before = g2.num_vertices()
            g2.add_edge_table(etab)
            g2.remove_edges(rem_edges)
            g2.remove_vertices(rem_verts)
            if check_isolated_vertices:
                g2.remove_isolated_vertices()
            queue.append(g2)
            after = g2.num_vertices()
            print("pivot ", before, after)

            _g2 = deepcopy(g2)
            circ = zx.extract_circuit(_g2)
            circ = zx.basic_optimization(circ)
            if circ.depth() < min_depth:
                min_depth = circ.depth()
                print(min_depth)
        except:
            pass
    for lcomp_match in lcomps:
        g2 = deepcopy(g1)
        etab, rem_verts, rem_edges, check_isolated_vertices = zx.rules.lcomp(g2, [lcomp_match])
                
        try:
            before = g2.num_vertices()
            g2.add_edge_table(etab)
            g2.remove_edges(rem_edges)
            g2.remove_vertices(rem_verts)
            if check_isolated_vertices:
                g2.remove_isolated_vertices()
            queue.append(g2)
            after = g2.num_vertices()
            print("lcomp ", before, after)

            _g2 = deepcopy(g2)
            circ = zx.extract_circuit(_g2)
            circ = zx.basic_optimization(circ)
            if circ.depth() < min_depth:
                min_depth = circ.depth()
                print(min_depth)
        except:
            pass






lcomp  40 39
lcomp  40 39
lcomp  40 39
lcomp  40 39
lcomp  40 39
Queue length: 4161
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
pivot  41 40
pivot  41 39
pivot  41 40
pivot  41 40
pivot  41 39
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
Queue length: 4176
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
pivot  41 39
pivot  41 40
pivot  41 40
pivot  41 39
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
Queue length: 4191
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
pivot  41 40
pivot  41 39
pivot  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
Queue length: 4205
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
pivot  41 40
pivot  41 39
pivot  41 40
pivot  41 39
pivot  41 40
pivot  41 39
lcomp  41 40
lcomp  41 40
lcomp  41 40
lcomp  41 40
Queue length: 4221
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
id  41 40
pivot  41 40
pivot 